In [ ]:
import numpy as np
from angular import Angular_dependence
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# get all possible_j from k1 and L
def get_possible_j(k1, L):
    j1 = abs(k1) - 0.5
    x = abs(j1 - L)
    j = []
    while abs(x - abs(j1 + L + 1)) > 1e-6  :
        j.append(x)
        x += 1
    return j # didn't consider the parity clebsh, but the 3j symbol will handle that.

# get all possible kappa from j
def get_possible_k(j):
    k = []
    for i in j:
        x = int(i + 0.5)
        k.append(x)
        k.append(-x)
    return k

#read potential, density..
def read_data(name):
    f = open(name)
    N = 801
    fx = np.zeros(N)
    y = np.zeros((N, 4))
    pos = 0
    for i in f:
        a, b, c , d, e = i.split()
        a,b,c,d,e = float(a), float(b), float(c), float(d), float(e)
        fx[pos] = a
        y[pos] = np.array([b,c,d,e])
        pos += 1
    return fx, y

#read_wavefunction to a dic
def read_wavefunction(lo, up):
    for k in range(lo, up + 1):
        if k!=0:
            #open the file with that wavefunction
            f = open('wavefunction' + str(k) + '.txt')
            fx = np.zeros(N)
            y = np.zeros((N,2))
            pos = 0
            for i in f:
                global m
                a, b, c, m = [float(ii) for ii in i.split()]
                fx[pos] = a
                y[pos] = np.array([b,c])
                pos += 1
            wavefunction_dic[k] = y.copy()


#this is not actually potential, but all the radial part
def Combine_wavefunction_up(k1, L, m, potential_up):
    j = get_possible_j(k1, L)         # all the possible j the original kappa and L kapple to 
    k2_group = get_possible_k(j)      # all the possible kappa2 from this j
    for k2 in k2_group:               
        if k2 in kprime_part_up:
            # potential is everything on the left, and we also have the upper part of the wavefucntion(radial)
            kprime_part_up[k2] += potential_up * Angular_dependence(m, k1, k2, L) * np.sqrt(4 * np.pi / (2 * L + 1))
        else:
            kprime_part_up[k2] = potential_up * Angular_dependence(m, k1, k2, L) * np.sqrt(4 * np.pi / (2 * L + 1))

def Combine_wavefunction_lo(k1, L, m, potential_lo):
    j = get_possible_j(k1, L)         # all the possible j the original kappa and L kapple to 
    k2_group = get_possible_k(j)      # all the possible kappa2 from this j
    for k2 in k2_group:
        if k2 in kprime_part_lo:
            # potential is everything on the left, and we also have the upper part of the wavefucntion(radial)
            kprime_part_lo[k2] += potential_lo * Angular_dependence(m, k1, k2, L) * np.sqrt(4 * np.pi / (2 * L + 1))
        else:
            kprime_part_lo[k2] = potential_lo * Angular_dependence(m, k1, k2, L) * np.sqrt(4 * np.pi / (2 * L + 1))        

In [ ]:
#test for the protons
#construct non-symmetric part
kmin = -11
kmax = +11
maxL = 4
E = (-53.9884 + 939) / 197.326
h = 0.025
N = 801z
fx = np.arange(0,20 + h,h)
second_d = np.array([1., -2., 1]) / h **2 
first_d = np.array([-1,0,1]) / (2 * h)
wavefunction_dic = {}
parameter = -1
dipole = fx * parameter * np.sqrt(3.0/4/np.pi) / 197.326

all_pt = []
for i in range(maxL):
    fx, pot = read_data('potential' + str(i) + '.txt')
    all_pt.append(pot.copy() / 197.326)

read_wavefunction(kmin, kmax)

# get list of kappa for a single solution
# for every k and for every L, find the poosible k prime
# construct a list for every k.
kprime_part_up = {}
kprime_part_lo = {}

for i in range(maxL):
    potential_up = - all_pt[i][:,0]  + all_pt[i][:,1] + 0.5 * all_pt[i][:,2] + all_pt[i][:,3]
    potential_lo = all_pt[i][:,0] + all_pt[i][:,1] + 0.5 * all_pt[i][:,2] + all_pt[i][:,3]
    if i == 0:
        potential_up = potential_up + 939/197.326
        potential_lo = potential_lo - 939/197.326
    if i == 1:
        # minus sign is for proton, and this dipole potential is in vector potential part
        potential_up = potential_up - dipole          
        potential_lo = potential_lo - dipole
    for k in range(kmin, kmax + 1):
        if k != 0:
#             print 2
            Combine_wavefunction_up(k, i, m, potential_up * wavefunction_dic[k][:,0] / fx)
            Combine_wavefunction_lo(k, i, m, potential_lo * wavefunction_dic[k][:,1] / fx)
    

In [ ]:
# construct the symmetric part
kpart_up = {}
kpart_lo = {}
for k in range(kmin,kmax + 1):
    if k!=0:
        kpart_up[k] = np.correlate(wavefunction_dic[k][:,1] / fx , first_d, 'same') - \
            ( k - 1 ) / fx**2 * wavefunction_dic[k][:,1] + E * wavefunction_dic[k][:,0] / fx
        kpart_lo[k] = - np.correlate(wavefunction_dic[k][:,0] / fx , first_d, 'same') - \
            ( k + 1 ) / fx**2 * wavefunction_dic[k][:,0] + E * wavefunction_dic[k][:,1] / fx
        

In [ ]:
plt.plot(fx, kprime_part_up[12],'r-',fx,fx * 0,'b-')

In [ ]:
print kpart_up.keys()
print kprime_part_up.keys()